In [1]:
import warnings
warnings.filterwarnings('ignore')   

import gym
from gym.wrappers import Monitor
import numpy as np
import tensorflow as tf
import numpy as np
import sys

if "../" not in sys.path:
    sys.path.append("../")
     

import plotting
from collections import deque, namedtuple

In [2]:
env = gym.envs.make("Breakout-v0")

valid_actions = [0, 1, 2, 3]

In [23]:

class StateProcessor:
    
    '''
    Process image - resize it and convert it to grayscale
    '''
    def __init__(self):
        with tf.variable_scope("state_processor"):
            
            self.input_state = tf.placeholder(shape=[210, 160, 3], 
                                              dtype=tf.uint8)
            self.output = tf.image.rgb_to_grayscale(self.input_state)
            # why were these dimensions 
            self.output = tf.image.crop_to_bounding_box(self.output,
                                                        34, 0, 160, 160)
            self.output = tf.image.resize_images(self.output,
                                                 [84, 84],
                                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)4
            self.output = tf.squeeze(self.output)

            
            
        
            
            
    
    


In [24]:
def make_epsilon_greedy_policy(q_value_estimator, n_actions):
    
    def policy(sess, observations, epsilon):
        A = (np.ones(n_actions, dtype=float) * epsilon) / n_actions
        q_values = q_value_estimator.predict(sess, np.expand_dims(observations, 0))[0]
        best_action = np.argmax(q_values)
        A[best_action] += (1.0-epsilon)
        return A

    return policy

In [25]:
def deep_q_learning(sess,
                    env,
                    q_estimator,
                    target_estimator,
                    state_processor,
                    num_episodes,
                    experiment_dir,
                    replay_memory_size=500000,
                    replay_memory_init_size=50000,
                    update_target_estimator_every=10000,
                    discount_factor=0.99,
                    epsilon_start=1.0,
                    epsilon_end=0.1,
                    epsilon_decay_steps=500000,
                    batch_size=32,
                    record_video_every=50):
    
    Transition = namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
    
    # replay memory queue
    replay_memory =  []
    
    # for status
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))
    
    # i don't know - system status health - check this out later?
    current_process = psutil.Process()
    
    # check points, model, monitor directories
    checkpoint_dir = os.path.join(experiment_dir, "checkpoints")
    checkpoint_path = os.path.join(checkpoint_dir, "model")
    monitor_path = os.path.join(experiment_dir, "monitor")
    
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(monitor_path):
        os.makedirs(monitor_path)
    
    
    saver = tf.train.Saver()
    # Loading from previous checkpoint if we had one
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        print("Loading model checkpoint {}...\n".format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)
        
        
    #  get global timestep
    total_t = sess.run(tf.contrib.framework.get_global_step())

    #calculating epsilon decays  - dividing the space in a number of steps
    epsilons = np.linspace(epsilon_start, epsilon_end, epsilon_decay_steps)
    
    
    #get the policy function
    policy = make_epsilon_greedy_policy(q_value_estimator, len(VALID_ACTIONS))
    
    
    #populate replay memory with initial experience
    state = env.reset()
    
    import ipdb
    ipdb.set_trace()
    
    state = state_processor.process(sess, state)
    state = np.stack([state] * 4, axis=2)
    
    for i in range(replay_memory_init_size):
        
        action_probs = policy(sess, state, epsilons[min(total_t, epsilon_decay_steps-1)])
        action = np.random_choice(np.arange(len(action_probs)), p = action_probs)
        next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
        
        next_state = state_processor.process(sess, next_state)
        
        
        
        
    
    
    
    
    
    
    

    

In [26]:
import os
import psutil

tf.reset_default_graph()

# Where we save our checkpoints and graphs
experiment_dir = os.path.abspath("./experiments/{}".format(env.spec.id))

# Create a glboal step variable
global_step = tf.Variable(0, name='global_step', trainable=False)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t, stats in deep_q_learning(sess,
                                    env,
                                    q_estimator=None,
                                    target_estimator=None,
                                    state_processor=None,
                                    experiment_dir=experiment_dir,
                                    num_episodes=10000,
                                    replay_memory_size=500000,
                                    replay_memory_init_size=50000,
                                    update_target_estimator_every=10000,
                                    epsilon_start=1.0,
                                    epsilon_end=0.1,
                                    epsilon_decay_steps=500000,
                                    discount_factor=0.99,
                                    batch_size=32):

        print("\nEpisode Reward: {}".format(stats.episode_rewards[-1]))

NameError: name 'q_value_estimator' is not defined